In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [58]:
cols = ['status', 'duration', 'credit_hist', 'purpose', 'credit_amt', 'savings', 'employment', 'installment_rate', 'personal_status', 'debtors', 'residencesince', 'property', 'age', 'install_plans', 'housing', 'existing_credits', 'job', 'maintenance_paying_people', 'telephone', 'foreign_worker', 'result']



In [59]:
df = pd.read_table('german.data', names=cols, sep=" ", index_col=False)

In [60]:
df.head()

,status,duration,credit_hist,purpose,credit_amt,savings,employment,installment_rate,personal_status,debtors,...,property,age,install_plans,housing,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [61]:
def preprocess_german(df): 
    df['status'] = df['status'].map({'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}).astype(int)
    df['credit_hist'] = df['credit_hist'].map({'A34': 0, 'A33': 1, 'A32': 2, 'A31': 3, 'A30': 4}).astype(int)
    df.loc[(df['credit_amt'] <= 2000), 'credit_amt'] = 0
    df.loc[(df['credit_amt'] > 2000) & (df['credit_amt'] <= 5000), 'credit_amt'] = 1
    df.loc[(df['credit_amt'] > 5000), 'credit_amt'] = 2   
    df.loc[(df['duration'] <= 12), 'duration'] = 0
    df.loc[(df['duration'] > 12) & (df['duration'] <= 24), 'duration'] = 1
    df.loc[(df['duration'] > 24) & (df['duration'] <= 36), 'duration'] = 2
    df.loc[(df['duration'] > 36), 'duration'] = 3
    df['age'] = df['age'].apply(lambda x : 1 if x >= 45 else 0) # 1 if old, 0 if young
    df['savings'] = df['savings'].map({'A61': 0, 'A62': 1, 'A63': 2, 'A64': 3, 'A65': 4}).astype(int)
    df['employment'] = df['employment'].map({'A71': 0, 'A72': 1, 'A73': 2, 'A74': 3, 'A75': 4}).astype(int)    
    df['gender'] = df['personal_status'].map({'A91': 1, 'A92': 0, 'A93': 1, 'A94': 1, 'A95': 0}).astype(int)
    df['debtors'] = df['debtors'].map({'A101': 0, 'A102': 1, 'A103': 2}).astype(int)
    df['property'] = df['property'].map({'A121': 3, 'A122': 2, 'A123': 1, 'A124': 0}).astype(int)        
    df['install_plans'] = df['install_plans'].map({'A141': 1, 'A142': 1, 'A143': 0}).astype(int)
    df['job'] = df['job'].map({'A171': 0, 'A172': 1, 'A173': 2, 'A174': 3}).astype(int)    
    df['telephone'] = df['telephone'].map({'A191': 0, 'A192': 1}).astype(int)
    df['foreign_worker'] = df['foreign_worker'].map({'A201': 1, 'A202': 0}).astype(int)
    pd.get_dummies(df, columns=['purpose', 'housing'], drop_first=True)
    
    return df



In [62]:
X = preprocess_german(df);

In [63]:
df.head()

,status,duration,credit_hist,purpose,credit_amt,savings,employment,installment_rate,personal_status,debtors,...,age,install_plans,housing,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
0,0,0,0,A43,0,4,4,4,A93,0,...,1,0,A152,2,2,1,1,1,1,1
1,1,3,2,A43,2,0,2,2,A92,0,...,0,0,A152,1,2,1,0,1,2,0
2,3,0,0,A46,1,0,3,2,A93,0,...,1,0,A152,1,1,2,0,1,1,1
3,0,3,2,A42,2,0,3,2,A93,2,...,1,0,A153,1,2,2,0,1,1,1
4,0,1,1,A40,1,0,2,3,A93,0,...,1,0,A153,2,2,2,0,1,2,1


In [64]:
df = df.drop(["purpose", "housing"], axis=1)

In [65]:
df = df.drop(["personal_status"], axis=1)

In [66]:
df.columns

Index(['status', 'duration', 'credit_hist', 'credit_amt', 'savings',
       'employment', 'installment_rate', 'debtors', 'residencesince',
       'property', 'age', 'install_plans', 'existing_credits', 'job',
       'maintenance_paying_people', 'telephone', 'foreign_worker', 'result',
       'gender'],
      dtype='object')

In [67]:
X = df[['status', 'duration', 'credit_hist', 'credit_amt', 'savings',
       'employment', 'installment_rate', 'debtors', 'residencesince',
       'property', 'age', 'install_plans', 'existing_credits', 'job',
       'maintenance_paying_people', 'telephone', 'foreign_worker',
       'gender']]

In [68]:
y = df[["result"]]

In [69]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=15)

In [70]:
x_train.columns

Index(['status', 'duration', 'credit_hist', 'credit_amt', 'savings',
       'employment', 'installment_rate', 'debtors', 'residencesince',
       'property', 'age', 'install_plans', 'existing_credits', 'job',
       'maintenance_paying_people', 'telephone', 'foreign_worker', 'gender'],
      dtype='object')

In [71]:
df.head()


,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
0,0,0,0,0,4,4,4,0,4,3,1,0,2,2,1,1,1,1,1
1,1,3,2,2,0,2,2,0,2,3,0,0,1,2,1,0,1,2,0
2,3,0,0,1,0,3,2,0,3,3,1,0,1,1,2,0,1,1,1
3,0,3,2,2,0,3,2,2,4,2,1,0,1,2,2,0,1,1,1
4,0,1,1,1,0,2,3,0,4,0,1,0,2,2,2,0,1,2,1


In [72]:
logistic_model = LogisticRegression()

In [73]:
logistic_model = LogisticRegression()

In [75]:
y.head()

,result
0,1
1,2
2,1
3,1
4,2


In [76]:
x_train

,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,gender
677,1,3,2,2,1,2,2,0,2,1,0,0,1,2,1,0,1,1
134,3,3,2,2,1,3,2,0,4,3,0,0,1,2,1,1,1,0
18,1,1,2,2,0,4,4,0,2,0,0,0,1,3,1,1,1,0
703,1,2,1,1,1,4,4,0,2,2,0,1,2,2,1,0,1,1
628,3,3,0,1,2,2,4,0,4,3,0,0,2,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,1,3,2,2,0,2,2,0,2,1,0,0,1,2,1,1,1,1
645,3,2,1,2,4,1,4,0,4,1,0,0,2,2,1,1,1,1
908,3,1,1,1,0,1,1,0,2,2,1,0,2,1,1,0,1,0
757,2,1,0,0,4,2,3,0,4,0,0,0,2,2,1,1,1,1


In [77]:
logistic_model.fit(x_train, y_train.values.ravel())


LogisticRegression()

In [78]:
y_pred = logistic_model.predict(x_test)

In [79]:
y_pred_prob = logistic_model.predict_proba(x_test)

In [80]:
y_pred_prob

array([[0.61641611, 0.38358389],
       [0.237293  , 0.762707  ],
       [0.95622377, 0.04377623],
       [0.78665571, 0.21334429],
       [0.73189391, 0.26810609],
       [0.90555302, 0.09444698],
       [0.78783284, 0.21216716],
       [0.13637803, 0.86362197],
       [0.37521792, 0.62478208],
       [0.27861568, 0.72138432],
       [0.69567144, 0.30432856],
       [0.89493247, 0.10506753],
       [0.26069436, 0.73930564],
       [0.51426348, 0.48573652],
       [0.50148944, 0.49851056],
       [0.85694778, 0.14305222],
       [0.85877304, 0.14122696],
       [0.64707801, 0.35292199],
       [0.74533553, 0.25466447],
       [0.95668641, 0.04331359],
       [0.29850067, 0.70149933],
       [0.88256718, 0.11743282],
       [0.54034283, 0.45965717],
       [0.52914067, 0.47085933],
       [0.86669026, 0.13330974],
       [0.94225034, 0.05774966],
       [0.76066589, 0.23933411],
       [0.885076  , 0.114924  ],
       [0.94984739, 0.05015261],
       [0.69228306, 0.30771694],
       [0.

In [81]:
accuracy_score(y_test, y_pred)

0.745

In [82]:
x_test.iloc[0]

status                       0
duration                     1
credit_hist                  0
credit_amt                   0
savings                      0
employment                   4
installment_rate             4
debtors                      0
residencesince               3
property                     1
age                          0
install_plans                0
existing_credits             2
job                          2
maintenance_paying_people    1
telephone                    1
foreign_worker               1
gender                       1
Name: 825, dtype: int64

In [84]:
groups_age =  df.groupby(df['age'])

In [85]:
protected_group_age_df = groups_age.get_group(1)

In [86]:
protected_group_age_df.iloc[1]
# According to the result it shows that the division is done correctly as age is coming as 1 for protected group

status                       3
duration                     0
credit_hist                  0
credit_amt                   1
savings                      0
employment                   3
installment_rate             2
debtors                      0
residencesince               3
property                     3
age                          1
install_plans                0
existing_credits             1
job                          1
maintenance_paying_people    2
telephone                    0
foreign_worker               1
result                       1
gender                       1
Name: 2, dtype: int64

In [87]:
non_protected_group_age_df = groups_age.get_group(0)

In [88]:
non_protected_group_age_df.iloc[1]
# According to the result it shows that the division is done correctly as age is coming as 0 for non-protected group

status                       3
duration                     2
credit_hist                  2
credit_amt                   2
savings                      4
employment                   2
installment_rate             2
debtors                      0
residencesince               4
property                     0
age                          0
install_plans                0
existing_credits             1
job                          1
maintenance_paying_people    2
telephone                    1
foreign_worker               1
result                       1
gender                       1
Name: 5, dtype: int64

In [89]:
y_pred = logistic_model.predict(x_test)

In [90]:
y_pred

array([1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1,
       2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1,
       1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1,
       1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1], dtype=int64)

In [91]:
# There are various fairness metrics defined in the 'Fairness Definitions Explained' paper based on the predicted probabilities and actual outcome

# 1. Test fairness -> P(Y=1|S=s, G=m) = P(Y=1|S=s, G=f), which can be used for age(protected and non-protected classes)
# 2. Well-calibration -> P(Y=1|S=s, G=m) = P(Y=1|S=s, G=f) = s, which can be used for age(protected and non-protected classes)
# 3. Balance for positive class -> E(S|Y=1,G=m) = E(S|Y=1,G=f), which can be used for age(protected and non-protected classes)
# 4. Balance for negative class -> E(S|Y=0,G=m) = E(S|Y=0,G=f), which can be used for age(protected and non-protected classes)

In [92]:
# Now, we need to find the P(ŷ = 1 | y = 1, G = 0) and P(ŷ = 1 | y = 1, G = 1) as G is the sensitive attribute here

In [93]:
# Find the average y_pred_proba where the actual outcome Y = 1 for the divided groups 

In [94]:
non_protected_group_age_df.head()

,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
1,1,3,2,2,0,2,2,0,2,3,0,0,1,2,1,0,1,2,0
5,3,2,2,2,4,2,2,0,4,0,0,0,1,1,2,1,1,1,1
7,1,2,2,2,0,2,2,0,2,1,0,0,1,3,1,1,1,1,1
9,1,2,0,2,0,0,4,0,2,1,0,0,2,3,1,0,1,2,1
10,1,0,2,0,0,1,3,0,1,1,0,0,1,2,1,0,1,2,0


In [95]:
non_protected_group_age_credresult_1_df = non_protected_group_age_df.groupby(non_protected_group_age_df['result']).get_group(1)

In [96]:
non_protected_group_age_credresult_1_df.head()

,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
5,3,2,2,2,4,2,2,0,4,0,0,0,1,1,2,1,1,1,1
7,1,2,2,2,0,2,2,0,2,1,0,0,1,3,1,1,1,1,1
12,1,0,2,0,0,2,1,0,1,1,0,0,1,2,1,1,1,1,0
14,0,1,2,0,0,2,2,0,4,1,0,0,1,2,1,0,1,1,0
17,0,2,4,2,4,1,2,0,3,1,0,1,3,2,1,0,1,1,1


In [97]:
#Drop the result column as the logistic regression model will accept 18 columns as input.
non_protected_group_age_credresult_1_df = non_protected_group_age_credresult_1_df.drop(["result"], axis=1)

In [98]:
# Need to find the predictive probability for the dataframe and then finally find the average
y_pred_non_protected_group_age_credresult_1 = logistic_model.predict(non_protected_group_age_credresult_1_df)

In [99]:
non_protected_group_age_credresult_1_df

,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,gender
5,3,2,2,2,4,2,2,0,4,0,0,0,1,1,2,1,1,1
7,1,2,2,2,0,2,2,0,2,1,0,0,1,3,1,1,1,1
12,1,0,2,0,0,2,1,0,1,1,0,0,1,2,1,1,1,0
14,0,1,2,0,0,2,2,0,4,1,0,0,1,2,1,0,1,0
17,0,2,4,2,4,1,2,0,3,1,0,1,3,2,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,0,2,2,1,0,0,4,0,3,2,0,0,1,3,1,1,1,1
995,3,0,2,0,0,3,3,0,4,3,0,0,1,1,1,0,1,0
996,0,2,2,1,0,2,4,0,4,2,0,0,1,3,1,1,1,1
997,3,0,2,0,0,4,4,0,4,1,0,0,1,2,1,0,1,1


In [100]:
y_pred_non_protected_group_age_credresult_1

array([1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [101]:
#avergae predicted probability came out to be 0.879781 for non protected group

In [102]:
protected_group_age_df.head()

,status,duration,credit_hist,credit_amt,savings,employment,installment_rate,debtors,residencesince,property,age,install_plans,existing_credits,job,maintenance_paying_people,telephone,foreign_worker,result,gender
0,0,0,0,0,4,4,4,0,4,3,1,0,2,2,1,1,1,1,1
2,3,0,0,1,0,3,2,0,3,3,1,0,1,1,2,0,1,1,1
3,0,3,2,2,0,3,2,2,4,2,1,0,1,2,2,0,1,1,1
4,0,1,1,1,0,2,3,0,4,0,1,0,2,2,2,0,1,2,1
6,3,1,2,1,2,4,3,0,4,2,1,0,1,2,1,0,1,1,1


In [103]:
protected_group_age_credresult_1_df = protected_group_age_df.groupby(protected_group_age_df['result']).get_group(1)

In [104]:
#Drop the result column as the logistic regression model will accept 18 columns as input.
protected_group_age_credresult_1_df = protected_group_age_credresult_1_df.drop(["result"], axis=1)

In [105]:
# Need to find the predict probability for the dataframe and then finally find the average
y_pred_protected_group_age_credresult_1 = logistic_model.predict(protected_group_age_credresult_1_df)

In [106]:
y_pred_protected_group_age_credresult_1

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [107]:
#avergae predicted probability came out to be 0.92053 for protected group

In [108]:
# Need to find the predict probability for the dataframe and then finally find the average
y_pred_protected_group_age_credresult_1 = logistic_model.predict_proba(protected_group_age_credresult_1_df)

In [109]:
y_pred_protected_group_age_credresult_1

array([[0.90745819, 0.09254181],
       [0.95231141, 0.04768859],
       [0.41987939, 0.58012061],
       [0.92296287, 0.07703713],
       [0.96827148, 0.03172852],
       [0.96214966, 0.03785034],
       [0.89865273, 0.10134727],
       [0.8170071 , 0.1829929 ],
       [0.97369167, 0.02630833],
       [0.95878593, 0.04121407],
       [0.83260629, 0.16739371],
       [0.98236382, 0.01763618],
       [0.57639591, 0.42360409],
       [0.61706136, 0.38293864],
       [0.83044698, 0.16955302],
       [0.90119233, 0.09880767],
       [0.664041  , 0.335959  ],
       [0.67756585, 0.32243415],
       [0.95262688, 0.04737312],
       [0.76716656, 0.23283344],
       [0.89493247, 0.10506753],
       [0.96017966, 0.03982034],
       [0.95685294, 0.04314706],
       [0.88176087, 0.11823913],
       [0.63052876, 0.36947124],
       [0.92381299, 0.07618701],
       [0.94545962, 0.05454038],
       [0.91818632, 0.08181368],
       [0.3930579 , 0.6069421 ],
       [0.56975585, 0.43024415],
       [0.

In [110]:
logistic_model.classes_

array([1, 2], dtype=int64)

In [115]:
y_pred_protected_group_age_credresult_1.transpose()[0].mean()

0.805612629449424

In [116]:
y_pred_non_protected_group_age_credresult_1 = logistic_model.predict_proba(non_protected_group_age_credresult_1_df)

In [117]:
y_pred_non_protected_group_age_credresult_1.transpose()[0].mean()

0.7551598695019986